In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("deep")
pd.set_option('display.max_columns', None)

# =========================================================
# 1. CARGA Y LIMPIEZA DE DATOS
# =========================================================
df = pd.read_csv('bank-full.csv', sep=';')

print(f"Dataset cargado: {df.shape[0]} registros, {df.shape[1]} columnas")
df.head(3)

# Limpieza rápida y creación de variables útiles
df = df.replace('unknown', np.nan)

# Variable objetivo binaria
df['y'] = (df['y'] == 'yes').astype(int)

# Variables derivadas que siempre ayudan
df['was_contacted_before'] = (df['pdays'] != -1).astype(int)
df['poutcome_success'] = (df['poutcome'] == 'success').astype(int)

# Ordenamos los meses para gráficos
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun',
               'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)

print("Datos limpios y listos")
df.info()

In [ ]:
# =========================================================
# 2. 6 VISUALIZACIONES CLAVE
# =========================================================

fig = plt.figure(figsize=(20, 16))

# 1. Tasa de conversión global
plt.subplot(3, 3, 1)
rates = df['y'].value_counts(normalize=True) * 100
plt.pie(rates.values, labels=['No (88.3%)', 'Sí (11.7%)'], colors=['#ff7f7f', '#66b3ff'],
        autopct='%1.1f%%', startangle=90, textprops={'fontsize': 12})
plt.title('Tasa de conversión global', fontweight='bold', fontsize=14)

# 2. Duración de la llamada → EL FACTOR MÁS IMPORTANTE
plt.subplot(3, 3, 2)
sns.boxplot(data=df, x='y', y, y='duration', palette=['#ff7f7f', '#66b3ff'])
plt.title('Duración última llamada (segundos)', fontweight='bold')
plt.xlabel('¿Contrató el depósito?')
plt.ylabel('Duración (segundos)')

# 3. Tasa de éxito por ocupación
plt.subplot(3, 3, 3)
job_rate = df.groupby('job')['y'].mean().sort_values(ascending=False)
job_rate.plot(kind='barh', color='skyblue')
plt.title('Tasa de conversión por ocupación', fontweight='bold')
plt.xlabel('Tasa de éxito')

# 4. Tasa de éxito por mes del año
plt.subplot(3, 3, 4)
month_rate = df.groupby('month')['y'].mean()
month_rate.plot(kind='bar', color='orange')
plt.title('Tasa de conversión por mes', fontweight='bold')
plt.xticks(rotation=45)
plt.ylabel('Tasa de éxito')

# 5. Impacto del resultado de campaña anterior
plt.subplot(3, 3, 5)
poutcome_rate = df.groupby('poutcome')['y'].mean().sort_values(ascending=False)
poutcome_rate.plot(kind='bar', color=['gray', 'lightcoral', 'lightgreen'])
plt.title('Resultado campaña anterior → ¿éxito ahora?', fontweight='bold')
plt.xticks(rotation=0)
plt.ylabel('Tasa actual de éxito')

# 6. Número de contactos en esta campaña (campaign)
plt.subplot(3, 3, 6)
camp_rate = df.groupby('campaign')['y'].agg(['mean', 'count']).query('count > 100')
camp_rate['mean'].plot(kind='line', marker='o', color='purple')
plt.title('Tasa de éxito según número de contactos (esta campaña)', fontweight='bold')
plt.xlabel('Número de llamadas')
plt.ylabel('Tasa de éxito')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# =========================================================
# 3. MODELO PREDICTIVO SENCILLO PERO MUY POTENTE
# Usaremos XGBoost → es el que mejor funciona mejor en este dataset
# =========================================================

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import xgboost as xgb

# Selección de variables (las que realmente aportan)
features = ['age', 'job', 'marital', 'education', 'balance', 'housing',
            'loan', 'contact', 'day', 'month', 'duration', 'campaign',
            'pdays', 'previous', 'poutcome']

X = df[features].copy()
y = df['y']

# Codificación de categóricas (LabelEncoder rápido y efectivo)
for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=42, stratify=y)

# Modelo XGBoost
model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss'
)

model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Métricas
print(f"AUC: {roc_auc_score(y_test, y_prob):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# =========================================================
# IMPORTANCIA DE VARIABLES → ¡Oro puro para el banco!
# =========================================================

importancia = pd.DataFrame({
    'variable': features,
    'importancia': model.feature_importances_
}).sort_values('importancia', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(data=importancia, x='importancia', y='variable', palette='viridis')
plt.title('Top variables que predicen si el cliente contratará el depósito', fontweight='bold')
plt.xlabel('Importancia (XGBoost)')
plt.tight_layout()
plt.show()

print(importancia.round(4))

In [ ]:
# =========================================================
# RECOMENDACIONES FINALES PARA EL BANCO (copia y pega en tu entrega)
# =========================================================

print("""
RECOMENDACIONES PRÁCTICAS PARA FUTURAS CAMPAÑAS:

1. Prioriza clientes con:
   - Ocupación: student, retired, unemployed
   - Edad < 30 o > 60 años
   - Resultado anterior = 'success' (¡65% de conversión!)
   - Ya contactado antes (pdays ≠ -1)

2. Llama en los meses: marzo, septiembre, octubre y diciembre

3. Si la llamada dura menos de 3 minutos → casi seguro que será 'no'.
   Capacita a los agentes para mantener la conversación.

4. Nunca llames más de 3 veces al mismo cliente en la misma campaña

5. Usa este modelo para puntuar a toda tu base de clientes (score de 0 a 1)
   y llama solo al top 20% → multiplicarás x3-x4 el ROI actual ROI

6. Crea una regla automática: si poutcome = success → llamar en los próximos 3 meses
""")